In [2]:
!pip install opencv-python numpy torch torchvision
!pip install yolov5  # If using YOLOv5 for detection


  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [1]:
pip install --upgrade sympy


  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [3]:
import os
import cv2
import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [4]:
image_folder = '/content/images'  # Path to the image folder
output_folder = '/content/annotated_data'  # Folder where annotated data will be saved

# Create output folder if it doesn't exist
Path(output_folder).mkdir(parents=True, exist_ok=True)

# Load the YOLOv5 model (you can also use other models if you prefer)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # YOLOv5 small model for airplane detection

# Define class names (YOLO label for airplane is usually 2)
class_names = model.names  # Dictionary mapping class IDs to class names
airplane_class_id = [key for key, value in class_names.items() if value == 'airplane'][0]


# Create the data.yaml file content
yaml_content = """
train: ./train/images
val: ./val/images

nc: 1  # Number of classes (Airplane in this case)
names: ['airplane']  # Class names
"""

# Write the data.yaml file
with open('data.yaml', 'w') as f:
    f.write(yaml_content)

# Function to convert bounding box coordinates to YOLO format
def convert_to_yolo_format(bbox, img_width, img_height):
    x_center = (bbox[0] + bbox[2]) / 2.0
    y_center = (bbox[1] + bbox[3]) / 2.0
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]

    # Normalize the coordinates
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return x_center, y_center, width, height

# Process the images
image_paths = list(Path(image_folder).rglob('*.jpg'))  # Assuming images are in jpg format
for image_path in tqdm(image_paths):
    # Read image
    img = cv2.imread(str(image_path))
    img_height, img_width, _ = img.shape

    # Detect objects in the image
    results = model(img)
    detections = results.xyxy[0].cpu().numpy()  # Get detections for the first image

    # Create annotation file
    annotation_path = Path(output_folder) / f'{image_path.stem}.txt'
    with open(annotation_path, 'w') as annot_file:
        for detection in detections:
            # Extract the class ID and bounding box
            class_id = int(detection[5])  # YOLO class ID (airplane class should be 2)
            if class_id == airplane_class_id:  # Only consider airplanes
                # Get the bounding box coordinates (xmin, ymin, xmax, ymax)
                bbox = detection[:4]

                # Convert the bounding box to YOLO format (x_center, y_center, width, height)
                x_center, y_center, width, height = convert_to_yolo_format(bbox, img_width, img_height)

                # Write to annotation file in YOLO format (class_id, x_center, y_center, width, height)
                annot_file.write(f"{airplane_class_id} {x_center} {y_center} {width} {height}\n")

    # Save the image to annotated_data folder
    output_image_path = Path(output_folder) / f'{image_path.name}'
    cv2.imwrite(str(output_image_path), img)

print("Annotation and labeling completed!")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-19 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
  0%|          | 0/4 [00:00<?, ?it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
 25%|██▌       | 1/4 [00:00<00:01,  2.10it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
 50%|█████     | 2/4 [00:00<00:00,  2.07it/s]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)

Annotation and labeling completed!
